In [1]:
# Dependencies
import requests
import json
import gmaps
import numpy as np
import pandas as pd

# Google developer API key
from config import gkey

In [2]:
# import the data we will use to analyze shops
shop_df = pd.read_csv("Resources/MedicallyEndorsedRetailers.csv", encoding="utf-8")

# Visualize
shop_df.dropna(how='all')
shop_df.head()

,License,UBI,Tradename,Privilege,Status,Med Privilege Code,Termination Code,Street Adress,Suite Rm,City,State,County,Zip Code,Date Created,Day Phone,Email
0,415470,6.029980e+15,2020 SOLUTIONS,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,2018 IRON ST STE A,,BELLINGHAM,WA,WHATCOM,982264212,20151012,3603938697,TLOZANO36277@YAHOO.COM
1,413314,6.034150e+15,2020 SOLUTIONS ON THE GUIDE,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,5655 GUIDE MERIDIAN STE A,,BELLINGHAM,WA,WHATCOM,982269722,20151010,3607342020,ACCOUNTING@RETAILBYCHAMPION.COM
2,84154,6.033600e+15,25 TREES,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,3213 S 38TH STE C,,TACOMA,WA,PIERCE,984041094,20180116,2533768562,25treesllc@gmail.com
3,422099,6.035980e+15,365 RECREATIONAL CANNABIS,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,36711 U.S. HIGHWAY 12,,DAYTON,WA,COLUMBIA,983288622,20170926,4259194183,seantherider@hotmail.com
4,423784,6.034300e+15,365 RECREATIONAL CANNABIS,MARIJUANA RETAILER,ACTIVE (ISSUED),395,,17517 15TH AVE NE #B,,SHORELINE,WA,KING,981553801,20160926,4259194183,seantherider@hotmail.com


In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [4]:
# Add columns for lat, lng
shop_df["Lat"] = ""
shop_df["Lng"] = ""
shop_df.columns

Index(['License', 'UBI', 'Tradename', 'Privilege', 'Status',
       'Med Privilege Code', 'Termination Code', 'Street Adress', 'Suite Rm',
       'City', 'State', 'County', 'Zip Code', 'Date Created', 'Day Phone',
       'Email', 'Lat', 'Lng'],
      dtype='object')

In [5]:
# remove unnecesary shop information 
shop_df = shop_df[["Tradename","Street Adress","City","State","County","Zip Code","Lat","Lng"]]
shop_df.tail()

,Tradename,Street Adress,City,State,County,Zip Code,Lat,Lng
284,WASHINGTON O G,5300 17TH AVE NW STE B,SEATTLE,WA,KING,981073815,,
285,WESTERN BUD,11401 RAINIER AVE S,SEATTLE,WA,KING,981783947,,
286,WESTSIDE420 RECREATIONAL,4503 OCEAN BEACH HWY STE 103,LONGVIEW,WA,COWLITZ,986325055,,
287,WHITE RABBIT RETAIL,15928 HWY 99,LYNNWOOD,WA,SNOHOMISH,980870000,,
288,WORLD OF WEED,3202 E PORTLAND AVE,TACOMA,WA,PIERCE,984044929,,


In [6]:
# create a params dict that will be updated with address in each iteration
params = {"key": gkey}

# Loop through the shop_df and run a lat/long search for each address
for index,row in shop_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    address = row["Street Adress"]
    city = row["City"]
    state = row["State"]
    
    # update address key value
    params['address'] = f"{address},{city},{state}"
    
    # make request
    shop_lat_lng = requests.get(base_url, params=params)
    
    # print the url
    # print(shop_lat_lng.url)
    
    #convert to json
    shop_lat_lng = shop_lat_lng.json()
    
    # add lat and lng to df
    shop_df.loc[index, "Lat"] = shop_lat_lng["results"][0]["geometry"]["location"]["lat"]
    shop_df.loc[index, "Lng"] = shop_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [8]:
# Visualize to confirm lat and lng
shop_df.tail()

,Tradename,Street Adress,City,State,County,Zip Code,Lat,Lng
284,WASHINGTON O G,5300 17TH AVE NW STE B,SEATTLE,WA,KING,981073815,47.6674,-122.379
285,WESTERN BUD,11401 RAINIER AVE S,SEATTLE,WA,KING,981783947,47.5009,-122.222
286,WESTSIDE420 RECREATIONAL,4503 OCEAN BEACH HWY STE 103,LONGVIEW,WA,COWLITZ,986325055,46.1625,-122.999
287,WHITE RABBIT RETAIL,15928 HWY 99,LYNNWOOD,WA,SNOHOMISH,980870000,47.8544,-122.292
288,WORLD OF WEED,3202 E PORTLAND AVE,TACOMA,WA,PIERCE,984044929,47.2345,-122.41


In [9]:
# create a list containing all coordinates
coordinates = []

for index,row in shop_df.iterrows():
    coordinates.append([row["Lat"],row["Lng"]])
    
coordinates

[[48.7570594, -122.4659928],
 [48.84634029999999, -122.4863025],
 [47.222817, -122.4788343],
 [46.3111813, -117.9943499],
 [47.7562484, -122.3137876],
 [47.64163139999999, -117.4886216],
 [46.9955685, -122.9069278],
 [47.04852, -122.7954805],
 [47.0044805, -123.4043625],
 [46.1456599, -122.929237],
 [47.04988849999999, -122.9949418],
 [47.04791549999999, -122.7960181],
 [47.0469316, -122.9255023],
 [47.5549709, -122.2901659],
 [47.7465702, -122.3459965],
 [47.8226234, -122.2064337],
 [46.061236, -118.346435],
 [46.5691867, -122.2669023],
 [48.1754335, -117.7254443],
 [47.7391728, -122.290312],
 [46.2197696, -119.7845671],
 [47.6743475, -122.1281923],
 [47.554262, -120.6026507],
 [48.46268329999999, -122.5814243],
 [47.1243041, -119.2904331],
 [47.7069133, -117.1169724],
 [47.94329200000001, -119.006346],
 [47.541448, -120.560674],
 [47.2438673, -123.0984389],
 [47.6166693, -122.1847115],
 [48.0328516, -122.7641775],
 [47.6568115, -122.6867313],
 [48.3005394, -117.7609423],
 [46.9998379

In [10]:
# Customize the size of the figure
figure_layout = {
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot the map
fig = gmaps.figure(layout=figure_layout)

# Create a layer
markers = gmaps.marker_layer(coordinates)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px'))